In [77]:
from ismn.interface import ISMN_Interface
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import re
import numpy as np
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [78]:
path=r'data\africa\Data_separate_files_header_20160614_20250614_11438_iyLN_20250614.zip'
# Load data and metadata
ismn_data=ISMN_Interface(path,parallel=True)
ismn_data

Using the existing ismn metadata in data\africa\python_metadata\Data_separate_files_header_20160614_20250614_11438_iyLN_20250614.csv to set up ISMN_Interface. 
If there are issues with the data reader, you can remove the metadata csv file to repeat metadata collection.


ismn.base.IsmnRoot Zip at data\africa\Data_separate_files_header_20160614_20250614_11438_iyLN_20250614.zip
with Networks[Stations]:
------------------------
  AMMA-CATCH: ['Banizoumbou', 'Belefoungou-Mid', 'Belefoungou-Top', 'Nalohou-Mid', 'Nalohou-Top', 'Tondikiboro', 'Wankama'],
  SD_DEM: ['Demokeya'],
  TAHMO: ['AtebubuSHS,Atebubu', 'BiaSHTSDebiso', 'BimbillaSHS,Bimbilla', 'BongoSHS,Bongo', 'BuiPowerAuthority,Bui', 'CRIG(SoilMoistureStation1)', 'CRIG(SoilMoistureStation2)', 'CSIR-SARI,Nyankpala-Tamale', 'CSIR-SARI,Tanina', 'CoteDIvoireMeteoOffice', 'DaffiamaSHS,Daffiama', 'DedanKimathiUniversityofTechnology', 'GbewaaCollegeofEducation,Pusiga', 'HanSHS,Han', 'ImurtottPrimaryschool', 'JuabosoSHS', 'KNUSTFarm,Kumasi', 'KajajiSHS,Kajaji', 'KpandaiSHS,Kpandai', 'MagomanoSecondarySchool', 'MahaliMzuri', 'MaraTriangleAirstrip', 'MasenoUniversity', 'MurungaruSecondarySchool', 'NDMA-TanaRiverHola', 'NaboishoCamp', 'NotreDameSeminary,SHS,Navrongo', 'SacredHeartSHSNsoatre', 'SenchuraSecondaryS

In [74]:
type(ismn_data.networks)

collections.OrderedDict

In [76]:
for b in ismn_data.networks:
    print(b)

AMMA-CATCH
SD_DEM
TAHMO


In [79]:
# Select 'Network' and 'Station', print metadata
sm=ismn_data['AMMA-CATCH'].to_xarray(variable='soil_moisture')
sm

100%|██████████| 7/7 [00:03<00:00,  1.96it/s]


<xarray.Dataset> Size: 9MB
Dimensions:                  (sensor: 16, date_time: 22344)
Coordinates:
  * date_time                (date_time) datetime64[ns] 179kB 2016-06-14 ... ...
Dimensions without coordinates: sensor
Data variables: (12/25)
    soil_moisture            (sensor, date_time) float64 3MB dask.array<chunksize=(1, 12348), meta=np.ndarray>
    soil_moisture_flag       (sensor, date_time) object 3MB dask.array<chunksize=(1, 12348), meta=np.ndarray>
    soil_moisture_orig_flag  (sensor, date_time) object 3MB dask.array<chunksize=(1, 12348), meta=np.ndarray>
    depth_from               (sensor) float64 128B dask.array<chunksize=(1,), meta=np.ndarray>
    depth_to                 (sensor) float64 128B dask.array<chunksize=(1,), meta=np.ndarray>
    clay_fraction            (sensor) float64 128B dask.array<chunksize=(1,), meta=np.ndarray>
    ...                       ...
    saturation               (sensor) float64 128B dask.array<chunksize=(1,), meta=np.ndarray>
    silt_fraction            (sensor) float64 128B dask.array<chunksize=(1,), meta=np.ndarray>
    station                  (sensor) <U15 960B dask.array<chunksize=(1,), meta=np.ndarray>
    timerange_from           (sensor) datetime64[ns] 128B dask.array<chunksize=(1,), meta=np.ndarray>
    timerange_to             (sensor) datetime64[ns] 128B dask.array<chunksize=(1,), meta=np.ndarray>
    variable                 (sensor) <U13 832B dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    n_sensors:   16
    n_stations:  7
    network:     AMMA-CATCH

In [ ]:
# ========================================
# 1. DATA EXPLORATION AND UNDERSTANDING
# ========================================

print("=== AMMA-CATCH Dataset Overview ===")
print(f"Dataset dimensions: {sm.dims}")
print(f"Number of sensors: {sm.dims['sensor']}")
print(f"Time range: {sm.date_time.min().values} to {sm.date_time.max().values}")
print(f"Total time points: {sm.dims['date_time']}")
print("\nData variables:")
for var in sm.data_vars:
    print(f"  - {var}: {sm[var].dims}")


=== AMMA-CATCH Dataset Overview ===
Dataset dimensions: FrozenMappingWarningOnValuesAccess({'sensor': 16, 'date_time': 22344})
Number of sensors: 16
Time range: 2016-06-14T00:00:00.000000000 to 2018-12-31T23:00:00.000000000
Total time points: 22344

Data variables:
  - soil_moisture: ('sensor', 'date_time')
  - soil_moisture_flag: ('sensor', 'date_time')
  - soil_moisture_orig_flag: ('sensor', 'date_time')
  - depth_from: ('sensor',)
  - depth_to: ('sensor',)
  - clay_fraction: ('sensor',)
  - climate_KG: ('sensor',)
  - climate_insitu: ('sensor',)
  - elevation: ('sensor',)
  - instrument: ('sensor',)
  - latitude: ('sensor',)
  - lc_2000: ('sensor',)
  - lc_2005: ('sensor',)
  - lc_2010: ('sensor',)
  - lc_insitu: ('sensor',)
  - longitude: ('sensor',)
  - network: ('sensor',)
  - organic_carbon: ('sensor',)
  - sand_fraction: ('sensor',)
  - saturation: ('sensor',)
  - silt_fraction: ('sensor',)
  - station: ('sensor',)
  - timerange_from: ('sensor',)
  - timerange_to: ('sensor',)
 

In [81]:
# 2. EXTRACT STATIC INFORMATION
# ========================================

print("\n=== Extracting Static Information ===")

# Create a dataframe with static information for each sensor
static_data = []

for sensor_idx in range(sm.dims['sensor']):
    sensor_data = {
        'sensor_id': sensor_idx,
        'network': sm.network.values[sensor_idx] if 'network' in sm.variables else 'AMMA-CATCH',
        'station': sm.station.values[sensor_idx] if 'station' in sm.variables else f'Station_{sensor_idx}',
        'sensor_name': str(sm.instrument.values[sensor_idx]) if 'instrument' in sm.variables else f'Sensor_{sensor_idx}',
        'latitude': float(sm.latitude.values[sensor_idx]),
        'longitude': float(sm.longitude.values[sensor_idx]),
        'elevation': float(sm.elevation.values[sensor_idx]) if 'elevation' in sm.variables else np.nan,
        'depth_from': float(sm.depth_from.values[sensor_idx]) if 'depth_from' in sm.variables else np.nan,
        'depth_to': float(sm.depth_to.values[sensor_idx]) if 'depth_to' in sm.variables else np.nan,
        'clay_fraction': float(sm.clay_fraction.values[sensor_idx]) if 'clay_fraction' in sm.variables else np.nan,
        'sand_fraction': float(sm.sand_fraction.values[sensor_idx]) if 'sand_fraction' in sm.variables else np.nan,
        'silt_fraction': float(sm.silt_fraction.values[sensor_idx]) if 'silt_fraction' in sm.variables else np.nan,
        'organic_carbon': float(sm.organic_carbon.values[sensor_idx]) if 'organic_carbon' in sm.variables else np.nan,
    }
    static_data.append(sensor_data)

static_df = pd.DataFrame(static_data)
print(f"Static data extracted for {len(static_df)} sensors")
print("\nSample of static data:")
print(static_df.head())



=== Extracting Static Information ===
Static data extracted for 16 sensors

Sample of static data:
   sensor_id     network          station sensor_name  latitude  longitude  \
0          0  AMMA-CATCH      Banizoumbou     CS616-1  13.53250    2.66040   
1          1  AMMA-CATCH      Banizoumbou     CS616-2  13.53250    2.66040   
2          2  AMMA-CATCH  Belefoungou-Mid       CS616   9.79506    1.70994   
3          3  AMMA-CATCH  Belefoungou-Mid       CS616   9.79506    1.70994   
4          4  AMMA-CATCH  Belefoungou-Top       CS616   9.78986    1.70994   

   elevation  depth_from  depth_to  clay_fraction  sand_fraction  \
0      208.0        0.05      0.05            5.0           90.0   
1      208.0        0.05      0.05            5.0           90.0   
2      414.0        0.05      0.05           22.0           49.0   
3      414.0        0.10      0.10           22.0           49.0   
4      414.0        0.05      0.05           22.0           49.0   

   silt_fraction  orga

In [82]:

# ========================================
# 3. CALCULATE DAILY MEAN, <AX, MIN, SD SOIL MOISTURE
# ========================================

print("\n=== Calculating Daily Mean Soil Moisture ===")

# Convert time to pandas datetime for easier resampling
sm_with_time = sm.assign_coords(date_time=pd.to_datetime(sm.date_time.values))

# Calculate daily means for soil moisture
daily_sm_mean = sm_with_time.soil_moisture.resample(date_time='D').mean()
print(f"Daily soil moisture data shape: {daily_sm_mean.shape}")
print(f"Date range: {daily_sm_mean.date_time.min().values} to {daily_sm_mean.date_time.max().values}")


# Calculate daily Max for soil moisture
daily_sm_max = sm_with_time.soil_moisture.resample(date_time='D').max()
print(f"Daily soil moisture data shape: {daily_sm_max.shape}")
print(f"Date range: {daily_sm_max.date_time.min().values} to {daily_sm_max.date_time.max().values}")

# Calculate daily Min for soil moisture
daily_sm_min = sm_with_time.soil_moisture.resample(date_time='D').min()
print(f"Daily soil moisture data shape: {daily_sm_min.shape}")
print(f"Date range: {daily_sm_min.date_time.min().values} to {daily_sm_min.date_time.max().values}")


# Calculate daily Std for soil moisture
daily_sm_sd = sm_with_time.soil_moisture.resample(date_time='D').std()
print(f"Daily soil moisture data shape: {daily_sm_sd.shape}")
print(f"Date range: {daily_sm_sd.date_time.min().values} to {daily_sm_sd.date_time.max().values}")




=== Calculating Daily Mean Soil Moisture ===
Daily soil moisture data shape: (16, 931)
Date range: 2016-06-14T00:00:00.000000000 to 2018-12-31T00:00:00.000000000
Daily soil moisture data shape: (16, 931)
Date range: 2016-06-14T00:00:00.000000000 to 2018-12-31T00:00:00.000000000
Daily soil moisture data shape: (16, 931)
Date range: 2016-06-14T00:00:00.000000000 to 2018-12-31T00:00:00.000000000
Daily soil moisture data shape: (16, 931)
Date range: 2016-06-14T00:00:00.000000000 to 2018-12-31T00:00:00.000000000


In [93]:
n_sensors=sm.dims['sensor']
n_sensors

16

In [94]:
# 4. CREATE DAILY TIME SERIES DATAFRAMES (OPTIMIZED)
# ========================================
print("\n=== Creating Daily Time Series DataFrames (Optimized) ===")

# Get date range for column names
dates = pd.to_datetime(daily_sm_mean.date_time.values)
date_strings = [d.strftime('%Y-%m-%d') for d in dates]

print(f"Processing {n_sensors} sensors for {len(date_strings)} days...")

# Extract all soil moisture data at once (vectorized operation)
sm_values = daily_sm_mean.values  # Shape: (n_days, n_sensors)

# Apply data quality filters vectorially
valid_mask = (
    (~np.isnan(sm_values)) & 
    (sm_values >= 0) & 
    (sm_values <= 1) & 
    (sm_values != -9999) & 
    (sm_values != -999)
)

# Apply mask - invalid values become NaN
sm_values_clean = np.where(valid_mask, sm_values, np.nan)

# Keep original shape (n_days, n_sensors) and transpose when creating DataFrame
# sm_values_clean shape is (n_days, n_sensors)

# Create static information columns
static_columns = ['sensor_id', 'network', 'station', 'sensor_name', 'latitude', 'longitude']
static_data = pd.DataFrame({
    'sensor_id': range(n_sensors),
    'network': static_df['network'].values,
    'station': static_df['station'].values,
    'sensor_name': static_df['sensor_name'].values,
    'latitude': static_df['latitude'].values,
    'longitude': static_df['longitude'].values
})

# Create date columns DataFrame - transpose to get (n_sensors, n_days)
date_data = pd.DataFrame(sm_values_clean, columns=date_strings)

# Combine static and date columns
daily_mean_df = pd.concat([static_data, date_data], axis=1)


=== Creating Daily Time Series DataFrames (Optimized) ===
Processing 16 sensors for 931 days...


In [95]:
# 4. CREATE DAILY TIME SERIES DATAFRAMES (OPTIMIZED)
# ========================================
print("\n=== Creating Daily Time Series DataFrames (Optimized) ===")

# Get date range for column names
dates = pd.to_datetime(daily_sm_max.date_time.values)
date_strings = [d.strftime('%Y-%m-%d') for d in dates]

print(f"Processing {n_sensors} sensors for {len(date_strings)} days...")

# Extract all soil moisture data at once (vectorized operation)
sm_values = daily_sm_max.values  # Shape: (n_days, n_sensors)

# Apply data quality filters vectorially
valid_mask = (
    (~np.isnan(sm_values)) & 
    (sm_values >= 0) & 
    (sm_values <= 1) & 
    (sm_values != -9999) & 
    (sm_values != -999)
)

# Apply mask - invalid values become NaN
sm_values_clean = np.where(valid_mask, sm_values, np.nan)

# Keep original shape (n_days, n_sensors) and transpose when creating DataFrame
# sm_values_clean shape is (n_days, n_sensors)

# Create static information columns
static_columns = ['sensor_id', 'network', 'station', 'sensor_name', 'latitude', 'longitude']
static_data = pd.DataFrame({
    'sensor_id': range(n_sensors),
    'network': static_df['network'].values,
    'station': static_df['station'].values,
    'sensor_name': static_df['sensor_name'].values,
    'latitude': static_df['latitude'].values,
    'longitude': static_df['longitude'].values
})

# Create date columns DataFrame - transpose to get (n_sensors, n_days)
date_data = pd.DataFrame(sm_values_clean, columns=date_strings)

# Combine static and date columns
daily_max_df = pd.concat([static_data, date_data], axis=1)


=== Creating Daily Time Series DataFrames (Optimized) ===
Processing 16 sensors for 931 days...


In [96]:
# 4. CREATE DAILY TIME SERIES DATAFRAMES (OPTIMIZED)
# ========================================
print("\n=== Creating Daily Time Series DataFrames (Optimized) ===")

# Get date range for column names
dates = pd.to_datetime(daily_sm_min.date_time.values)
date_strings = [d.strftime('%Y-%m-%d') for d in dates]

print(f"Processing {n_sensors} sensors for {len(date_strings)} days...")

# Extract all soil moisture data at once (vectorized operation)
sm_values = daily_sm_min.values  # Shape: (n_days, n_sensors)

# Apply data quality filters vectorially
valid_mask = (
    (~np.isnan(sm_values)) & 
    (sm_values >= 0) & 
    (sm_values <= 1) & 
    (sm_values != -9999) & 
    (sm_values != -999)
)

# Apply mask - invalid values become NaN
sm_values_clean = np.where(valid_mask, sm_values, np.nan)

# Keep original shape (n_days, n_sensors) and transpose when creating DataFrame
# sm_values_clean shape is (n_days, n_sensors)

# Create static information columns
static_columns = ['sensor_id', 'network', 'station', 'sensor_name', 'latitude', 'longitude']
static_data = pd.DataFrame({
    'sensor_id': range(n_sensors),
    'network': static_df['network'].values,
    'station': static_df['station'].values,
    'sensor_name': static_df['sensor_name'].values,
    'latitude': static_df['latitude'].values,
    'longitude': static_df['longitude'].values
})

# Create date columns DataFrame - transpose to get (n_sensors, n_days)
date_data = pd.DataFrame(sm_values_clean, columns=date_strings)

# Combine static and date columns
daily_min_df = pd.concat([static_data, date_data], axis=1)


=== Creating Daily Time Series DataFrames (Optimized) ===
Processing 16 sensors for 931 days...


In [97]:
# 4. CREATE DAILY TIME SERIES DATAFRAMES (OPTIMIZED)
# ========================================
print("\n=== Creating Daily Time Series DataFrames (Optimized) ===")

# Get date range for column names
dates = pd.to_datetime(daily_sm_sd.date_time.values)
date_strings = [d.strftime('%Y-%m-%d') for d in dates]

print(f"Processing {n_sensors} sensors for {len(date_strings)} days...")

# Extract all soil moisture data at once (vectorized operation)
sm_values = daily_sm_sd.values  # Shape: (n_days, n_sensors)

# Apply data quality filters vectorially
valid_mask = (
    (~np.isnan(sm_values)) & 
    (sm_values >= 0) & 
    (sm_values <= 1) & 
    (sm_values != -9999) & 
    (sm_values != -999)
)

# Apply mask - invalid values become NaN
sm_values_clean = np.where(valid_mask, sm_values, np.nan)

# Keep original shape (n_days, n_sensors) and transpose when creating DataFrame
# sm_values_clean shape is (n_days, n_sensors)

# Create static information columns
static_columns = ['sensor_id', 'network', 'station', 'sensor_name', 'latitude', 'longitude']
static_data = pd.DataFrame({
    'sensor_id': range(n_sensors),
    'network': static_df['network'].values,
    'station': static_df['station'].values,
    'sensor_name': static_df['sensor_name'].values,
    'latitude': static_df['latitude'].values,
    'longitude': static_df['longitude'].values
})

# Create date columns DataFrame - transpose to get (n_sensors, n_days)
date_data = pd.DataFrame(sm_values_clean, columns=date_strings)

# Combine static and date columns
daily_sd_df = pd.concat([static_data, date_data], axis=1)


=== Creating Daily Time Series DataFrames (Optimized) ===
Processing 16 sensors for 931 days...


In [98]:
print(f"Daily mean dataframe shape: {daily_mean_df.shape}")
print(f"Daily max dataframe shape: {daily_max_df.shape}")
print(f"Daily min dataframe shape: {daily_min_df.shape}")
print(f"Daily std dataframe shape: {daily_sd_df.shape}")



Daily mean dataframe shape: (16, 937)
Daily max dataframe shape: (16, 937)
Daily min dataframe shape: (16, 937)
Daily std dataframe shape: (16, 937)


In [100]:
daily_sd_df

,sensor_id,network,station,sensor_name,latitude,longitude,2016-06-14,2016-06-15,2016-06-16,2016-06-17,...,2018-12-22,2018-12-23,2018-12-24,2018-12-25,2018-12-26,2018-12-27,2018-12-28,2018-12-29,2018-12-30,2018-12-31
0,0,AMMA-CATCH,Banizoumbou,CS616-1,13.53250,2.66040,0.000343,0.000729,0.000325,0.000356,...,0.000310,0.000383,0.000365,0.000371,0.000359,0.000471,0.000431,0.000375,0.000278,0.000298
1,1,AMMA-CATCH,Banizoumbou,CS616-2,13.53250,2.66040,0.000381,0.000817,0.000374,0.000299,...,0.000357,0.000342,0.000332,0.000317,0.000339,0.000367,0.000354,0.000315,0.000326,0.000300
2,2,AMMA-CATCH,Belefoungou-Mid,CS616,9.79506,1.70994,0.001613,0.066359,0.017034,0.006777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,AMMA-CATCH,Belefoungou-Mid,CS616,9.79506,1.70994,0.000367,0.051490,0.014935,0.006591,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,AMMA-CATCH,Belefoungou-Top,CS616,9.78986,1.70994,0.000981,0.050848,0.012136,0.009025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,AMMA-CATCH,Belefoungou-Top,CS616,9.78986,1.70994,0.000220,0.020614,0.005535,0.005748,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,AMMA-CATCH,Nalohou-Mid,CS616,9.74530,1.60530,0.002031,0.042988,0.023123,0.010592,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,AMMA-CATCH,Nalohou-Mid,CS616,9.74530,1.60530,0.000704,0.014771,0.013111,0.007905,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,AMMA-CATCH,Nalohou-Top,CS616,9.74407,1.60580,0.000932,0.025147,0.011145,0.004752,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,AMMA-CATCH,Nalohou-Top,CS616,9.74407,1.60580,0.001017,0.005421,0.006179,0.004647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
